# Scrape BC Liquor Store Website

### Import libraries

In [4]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import re
from tqdm import tqdm
import csv

### Functions to scrape information from the website

In [5]:
def find_tags(var,begin,end):
    price =[]
    name=[]
    page =[]
    country =[]
    sku_link = []
    SCROLL_PAUSE_TIME = 5
    driver = webdriver.Chrome('C:/Users/Hai Ninh/Python Learning/chromedriver')
    
    for pagenum in tqdm(range(begin,end)):
        driver.get('https://www.bcliquorstores.com/product-catalogue?category=%s&sort=name.raw:asc&page=%d' % (var,pagenum))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        
        name_i = [i.get_text() for i in soup.find_all('h3', class_ = "product-name")]
        country_i = [i.get_text() for i in soup.find_all('span', class_ = "country-code")]
        sku_link_i = [i.get('href') for i in soup.find_all('a')][118:214:4]
        price_i= [i.get_text() for i in soup.find_all('div', class_ = "product-price")]
        
        #append to the final list
        name.append(name_i)
        country.append(country_i)
        sku_link.append(sku_link_i)
        price.append(price_i)
    
    return name, country, sku_link, price

In [12]:
name, country, sku_link, price = find_tags('beer', 1, 22)
name = [i for m in name for i in m]
country = [i for m in country for i in m]
sku_link= [i for m in sku_link for i in m][:-3]
price= [i for m in price for i in m]

100%|██████████| 21/21 [02:00<00:00,  5.74s/it]


In [15]:
df = pd.DataFrame({"name": name, "country":country, "price": price, "sku_link": sku_link})

In [16]:
df = pd.read_csv('bcstores.csv')
df.head()

,name,country,price,sku_link
0,33 ACRES OF LIFE LAGER,Canada,$12.29,/product/772335
1,33 ACRES OF NIRVANA 6 PACK,Canada,$13.49,/product/880930
2,33 ACRES OF OCEAN PALE ALE,Canada,$12.29,/product/837591
3,33 ACRES OF SUNSHINE BLANCHE,Canada,$12.29,/product/90837
4,33 ACRES OF SUNSHINE CAN,Canada,$21.99,/product/23859


In [19]:
def product_details(links):
   
    alc = []
    style = []
    beer_type =[]
    desc = []
    driver = webdriver.Chrome('C:/Users/Hai Ninh/Python Learning/chromedriver')
    
    for sku in tqdm(links):
        driver.get('https://www.bcliquorstores.com%s' % (sku))
        time.sleep(5)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        alc_i = [i.get_text() for i in soup.find_all('span', attrs={"data-bind": "text: _source.alcoholPercentage + '%'" })]
        style_i = [i.get_text() for i in soup.find_all('span', attrs={"data-bind": "text: _source.style" })]
        beer_type_i = [i.get_text() for i in soup.find_all('span', attrs={"data-bind": "text: $root.FormatProductType($data._source)" })]
        desc_i = [i.get_text() for i in soup.find_all('span', attrs={"data-bind": "text: _source.tastingDescription" })]
        
        #append to the final list
        alc.append(alc_i)
        style.append(style_i)
        beer_type.append(beer_type_i)
        desc.append(desc_i)
    
    return alc,style,beer_type,desc

In [20]:
alc,style,beer_type,desc = product_details(df.sku_link)

100%|██████████| 501/501 [48:10<00:00,  5.77s/it]


In [73]:
# alcohol by volume
df['alc'] = [i for m in alc for i in m][0::2]

# beer style
style_final = []

for i in style:
    if i == []:
        style_final.append(['No info','No info'])
    else: 
        style_final.append(i)

df['style'] = [i for m in style_final for i in m][0::2]

# beer type
df['beer_type'] = [i for m in beer_type for i in m][0::2]

#description
desc_final = []

for i in desc:
    if i == []:
        desc_final.append(['No info','No info'])
    else: 
        desc_final.append(i)
        
df['desc'] = [i for m in desc_final for i in m][0::2]

In [74]:
df.head()

,name,country,price,sku_link,alc,style,beer_type,desc
0,33 ACRES OF LIFE LAGER,Canada,$12.29,/product/772335,4.8%,California Common Or Steam,Domestic Beer - BC Craft Beer,"1848, West Coast convictions of resourcefulnes..."
1,33 ACRES OF NIRVANA 6 PACK,Canada,$13.49,/product/880930,7%,India Pale Ale,Domestic Beer - BC Craft Beer,33 Acres is one of Vancouver’s most well-loved...
2,33 ACRES OF OCEAN PALE ALE,Canada,$12.29,/product/837591,5.6%,Pale Ale,Domestic Beer - BC Craft Beer,A full flavoured beer integrated with a distin...
3,33 ACRES OF SUNSHINE BLANCHE,Canada,$12.29,/product/90837,5%,Wheat,Domestic Beer - BC Craft Beer,"Thanks to summers' launching acclaims, this ro..."
4,33 ACRES OF SUNSHINE CAN,Canada,$21.99,/product/23859,5%,Wheat,Domestic Beer - BC Craft Beer,No info


In [76]:
# df.to_csv('bcstores.csv', index=False)